In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

# 初始化权重
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x,W):
    # filter / kernel tensor of shape [filter_height,filter_width, in_channels, out_channels]
    # strides[1]: x方向的步长，strides[2]: y方向的步长
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

# 池化层
def max_pool_2x2(x):
    # ksize [1,x,y,1] 窗口大小
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [4]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])

# 改变x为3d的向量 [batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从一个平面获取特征
b_conv1 = bias_variable([32]) #每一个卷积核一个偏置值

# 把x_image和权值向量进行卷积，再加上偏置值，然后应用relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)  #卷积后是28*28*32
h_pool1 = max_pool_2x2(h_conv1) #池化后是14*14*32

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5,5,32,64]) # 5*5的采样窗口，64个卷积核从32个平面获取特征
b_conv2 = bias_variable([64]) #每一个卷积核一个偏置值

# 把h_pool1和权值向量进行卷积，再加上偏置值，然后应用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)  # 卷积后是14*14*64
h_pool2 = max_pool_2x2(h_conv2)  # 池化后是7*7*64

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])

# 初始化第一层1024个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024]) # 上一层有7*7*64个神经元
b_fc1 = bias_variable([1024])

# 求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2  = bias_variable([10])

# 计算输出
predictions = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=predictions))

# Optimizer
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)

# 结果
correct_prediction = tf.equal(tf.argmax(predictions,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:1.0})
        print("train step:", epoch, "test_acc:", acc)

 0 test_acc: 0.9567
train step: 1 test_acc: 0.9702
train step: 2 test_acc: 0.9763
train step: 3 test_acc: 0.9799
train step: 4 test_acc: 0.9816
train step: 5 test_acc: 0.9833
train step: 6 test_acc: 0.9859
train step: 7 test_acc: 0.9858
train step: 8 test_acc: 0.9892
train step: 9 test_acc: 0.9891
train step: 10 test_acc: 0.9906
